In [5]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
dataset = pd.read_csv('data.csv')
dataset = pd.read_csv('data.csv', parse_dates=["Date"])
dataset.columns = ["Date", "Start Date", "Visitor Team", "VisitorPts", "Home Team", "HomePts", "Box Score", "OT?", "Attend", "Notes"]
dataset["HomeWin"] = dataset["VisitorPts"] <dataset["HomePts"]
y_true = dataset["HomeWin"].values
encoding = LabelEncoder()
encoding.fit(dataset["Home Team"].values)
home_team = encoding.transform(dataset["Home Team"].values)
visitor_team = encoding.transform(dataset["Visitor Team"].values)
x_team = np.vstack([home_team,visitor_team]).T

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(random_state=14)
score = cross_val_score(clf, x_team, y_true, scoring='accuracy')
print("{0:.1f}".format(np.mean(score)*100))



55.5


In [8]:
from collections import defaultdict
dataset["HomeLastWin"] = False
dataset["VisitorLastWin"] = False
from _collections import defaultdict
won_last = defaultdict(int)
for index, row in dataset.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    row["HomeLastWin"] = won_last[home_team]
    row["VisitorLastWin"] = won_last[visitor_team]
    dataset.loc[index] = row
    won_last[home_team] = row["HomeWin"]
    won_last[visitor_team] = not row["HomeWin"]

last_match_winner = defaultdict(int)
dataset["HomeTeamWonLast"] = 0
for index,row in dataset.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    teams = tuple(sorted([home_team,visitor_team]))
    row["HomeTeamWonLast"] = 1 if last_match_winner[teams] == row["Home Team"] else 0
    dataset.loc[index] = row
    winner = row["Home Team"] if row["HomeWin"] else row["Visitor Team"]
    last_match_winner[teams] = winner
x_lastwinner = dataset[["HomeLastWin","VisitorLastWin","HomeTeamWonLast"]].values

In [9]:
x_all = np.hstack([x_lastwinner,x_team])
clf = RandomForestClassifier(random_state=14)
score = cross_val_score(clf, x_all, y_true, scoring='accuracy')
print("{0:.1f}".format(np.mean(score)*100))


58.7


In [11]:
from sklearn.model_selection import GridSearchCV
parameter_space = {
    "max_features":[2, 10, 'auto'],
    "n_estimators":[100,],
    "criterion":["gini", "entropy"],
    "min_samples_leaf":[2, 4, 6],
}
clf = RandomForestClassifier(random_state=14)
grid = GridSearchCV(clf,parameter_space)
grid.fit(x_all, y_true)
print("{0:.1f}".format(grid.best_score_*100))

/Users/yangbao/PycharmProjects/datadig/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/yangbao/PycharmProjects/datadig/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/yangbao/PycharmProjects/datadig/venv/lib/python3.7/site-packages/sklearn/ensemble/_forest.py", line 392, in fit
    for i, t in enumerate(trees))
  File "/Users/yangbao/PycharmProjects/datadig/venv/lib/python3.7/site-packages/joblib/parallel.py", line 1029, in __call__
    if self.dispatch_one_batch(iterator):
  File "/Users/yangbao/PycharmProjects/datadig/venv/lib/python3.7/site-packages/joblib/parallel.py", line 847, in dispatch_one_batch
    self._dispatch(tasks)
  File "/Users/yangbao/PycharmProj

62.4


In [12]:
print(grid.best_estimator_)



RandomForestClassifier(max_features=2, min_samples_leaf=4, random_state=14)
